In [2]:
import pandas as pd

In [3]:
df_label = pd.read_csv('dataset/operations_labels_training.csv')
df_train=pd.read_csv("dataset/telemetry_for_operations_training.csv")
valid = pd.read_csv('dataset/telemetry_for_operations_validation.csv')

In [5]:
# Check if the format is 'YYYY-MM-DD HH:MM:SS'
def check_format(date_str):
    # Try to convert to datetime with a specific format
    try:
        pd.to_datetime(date_str, format='%Y-%m-%d %H:%M:%S')
        return 'datetime'
    except ValueError:
        try:
            pd.to_datetime(date_str, format='%Y-%m-%d')
            return 'date'
        except ValueError:
            return 'invalid'

# Apply the function to the 'create_dt' column
df_train['format'] = df_train['create_dt'].apply(check_format)
# Display the rows with 'date' or 'invalid' format
df_train=df_train.drop(df_train[df_train['format'] != 'datetime'].index)
def assign_labels(group):
    object_labels = df_label[df_label['mdm_object_name'] == group.name]
    
    for _, row in object_labels.iterrows():
        mask = (group['create_dt'] > row['start_time']) & (group['create_dt'] < row['end_time'])
        group.loc[mask, 'operation_kind_id'] = row['operation_kind_id']
        group.loc[mask, 'name'] = row['name']
    
    return group

# Apply the function to each group of mdm_object_name
train = df_train.groupby('mdm_object_name').apply(assign_labels)

# Verify the results
print(train['operation_kind_id'].value_counts(dropna=False))

# Save the updated DataFrame
train.to_csv('train_with_labels.csv', index=False)

/tmp/ipykernel_38375/3574577746.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train = df_train.groupby('mdm_object_name').apply(assign_labels)


operation_kind_id
2.0    235243
5.0    162838
0.0     69217
1.0     30780
NaN     12968
3.0      5016
Name: count, dtype: int64


In [ ]:
import pandas as pd 
import numpy as np

#valid = pd.read_csv('dataset/telemetry_for_operations_validing.csv')
valid = pd.read_csv('dataset/telemetry_for_operations_validation.csv')



def safe_to_datetime(date_str):
    try:
        return pd.to_datetime(date_str)
    except:
        return pd.to_datetime(date_str, format='%Y-%m-%d')

# Convert timestamp columns to datetime
valid['create_dt'] = valid['create_dt'].apply(safe_to_datetime)
df_label['start_time'] = pd.to_datetime(df_label['start_time'])
df_label['end_time'] = pd.to_datetime(df_label['end_time'])

# As 4 is not assigned to anyone i m assing it invalid operation_kind_id
valid['operation_kind_id'] = 4

#datetime_mask = valid['create_dt'].notna() & (valid['create_dt'].dt.time != pd.Timestamp('00:00:00').time())

# Sort both DataFrames by mdm_object_name and timestamp for efficient processing
valid = valid.sort_values(['mdm_object_name', 'create_dt'])
df_label = df_label.sort_values(['mdm_object_name', 'start_time'])

def assign_operation_kind_id(group):
    object_df_labels = df_label[df_label['mdm_object_name'] == group.name]
    
    for _, row in object_df_labels.iterrows():
        mask = (group['create_dt'] > row['start_time']) & (group['create_dt'] < row['end_time']) #& datetime_mask
        group.loc[mask, 'operation_kind_id'] = row['operation_kind_id']
    
    return group

valid = valid.groupby('mdm_object_name').apply(assign_operation_kind_id)

print(valid['operation_kind_id'].value_counts(dropna=False))

# Optional: Check the percentage of date-only entries (where operation_kind_id is still 4)
date_only_percentage = (valid['operation_kind_id'] == 4).mean() * 100
print(f"Percentage of date-only entries: {date_only_percentage:.2f}%")

valid.to_csv('valid_with_operation_kind_id.csv', index=False)